## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [3]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal(shape=(784, 256)), trainable=True)
        self.W2 = tf.Variable(tf.random.normal(shape=(256, 10)), trainable=True)
        self.b1 = tf.Variable(tf.random.normal(shape=(256,)), trainable=True)
        self.b2 = tf.Variable(tf.random.normal(shape=(10,)), trainable=True)
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x_reshape = tf.reshape(x, shape=(x.shape[0], 784))

        logits = tf.matmul(tf.nn.softmax(tf.matmul(x_reshape, self.W1) + self.b1), self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [5]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [6]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.7430954 ; accuracy 0.09596667
epoch 1 : loss 2.7424603 ; accuracy 0.09616667
epoch 2 : loss 2.7418265 ; accuracy 0.09618333
epoch 3 : loss 2.7411945 ; accuracy 0.09628333
epoch 4 : loss 2.740564 ; accuracy 0.096366666
epoch 5 : loss 2.7399354 ; accuracy 0.09645
epoch 6 : loss 2.739308 ; accuracy 0.096483335
epoch 7 : loss 2.7386823 ; accuracy 0.0965
epoch 8 : loss 2.738058 ; accuracy 0.09685
epoch 9 : loss 2.7374353 ; accuracy 0.09705
epoch 10 : loss 2.736814 ; accuracy 0.09706666
epoch 11 : loss 2.7361948 ; accuracy 0.097216666
epoch 12 : loss 2.7355766 ; accuracy 0.097566664
epoch 13 : loss 2.7349598 ; accuracy 0.097783335
epoch 14 : loss 2.7343447 ; accuracy 0.09786667
epoch 15 : loss 2.7337313 ; accuracy 0.098
epoch 16 : loss 2.7331192 ; accuracy 0.098183334
epoch 17 : loss 2.7325087 ; accuracy 0.09836667
epoch 18 : loss 2.7318993 ; accuracy 0.098516665
epoch 19 : loss 2.7312913 ; accuracy 0.09861667
epoch 20 : loss 2.7306855 ; accuracy 0.09876667
epoch 21 : loss 2